# Get Returns 

In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta


dates = pd.read_csv('dates/dates_updated.csv')
dates = dates.drop(index=[59, 67, 161, 163, 164, 168]).reset_index(drop=True)
dates_new = dates
dates_new.to_csv('dates/dates_new.csv')


In [2]:
dates_statements = dates_new.copy()
dates_statements['Statement Date'] = pd.to_datetime(dates_statements['Statement Date'], errors='coerce')
dates_statements['document_type'] = 'statement'

dates_intermeetings = dates_new.copy()
dates_intermeetings['Intermeeting Date'] = pd.to_datetime(dates_intermeetings['Intermeeting Date'], errors='coerce')
dates_intermeetings['document_type'] = 'intermeeting'


/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_22308/3968209943.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates_statements['Statement Date'] = pd.to_datetime(dates_statements['Statement Date'], errors='coerce')
/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_22308/3968209943.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates_intermeetings['Intermeeting Date'] = pd.to_datetime(dates_intermeetings['Intermeeting Date'], errors='coerce')


In [3]:
dates_statements

,Statement Date,Intermeeting Date,Chair,document_type
0,2000-02-02,3/23/00,Alan Greenspan,statement
1,2000-03-21,5/18/00,Alan Greenspan,statement
2,2000-05-16,6/29/00,Alan Greenspan,statement
3,2000-06-28,8/24/00,Alan Greenspan,statement
4,2000-08-22,10/5/00,Alan Greenspan,statement
...,...,...,...,...
195,2024-09-18,10/9/24,Jerome Powell,statement
196,2024-11-07,11/26/24,Jerome Powell,statement
197,2024-12-18,1/8/25,Jerome Powell,statement
198,2025-01-29,2/19/25,Jerome Powell,statement


In [4]:
tickers = [
    '^GSPC', '^IXIC', '^DJI', '^RUT', '^W5000',
    'XLF', 'XLRE', 'XLU', 'XLY', 'XLP',
    'XLE', 'XLV', 'XLI', 'XLB', 'XLK', 'XLC',
    '^IRX', '^TNX'
]


In [5]:
dates_new

,Statement Date,Intermeeting Date,Chair
0,2/2/00,3/23/00,Alan Greenspan
1,3/21/00,5/18/00,Alan Greenspan
2,5/16/00,6/29/00,Alan Greenspan
3,6/28/00,8/24/00,Alan Greenspan
4,8/22/00,10/5/00,Alan Greenspan
...,...,...,...
195,9/18/24,10/9/24,Jerome Powell
196,11/7/24,11/26/24,Jerome Powell
197,12/18/24,1/8/25,Jerome Powell
198,1/29/25,2/19/25,Jerome Powell


In [6]:

dates['Statement Date'] = pd.to_datetime(dates['Statement Date'])
dates['document_type'] = 'statement' 

tickers = [
    '^GSPC',     # S&P 500
    '^IXIC',     # NASDAQ Composite
    '^DJI',      # Dow Jones Industrial Average
    '^RUT',      # Russell 2000
    '^W5000',    # Wilshire 5000
    'XLF',       # Financials Sector (ETF)
    'XLRE',      # Real Estate Sector (ETF)
    'XLU',       # Utilities Sector (ETF)
    'XLY',       # Consumer Discretionary Sector (ETF)
    'XLP',       # Consumer Staples Sector (ETF)
    'XLE',       # Energy Sector (ETF)
    'XLV',       # Healthcare Sector (ETF)
    'XLI',       # Industrials Sector (ETF)
    'XLB',       # Materials Sector (ETF)
    'XLK',       # Information Technology Sector (ETF)
    'XLC',       # Communication Services Sector (ETF)
    '^IRX',      # Three-month Treasury Bill Yield
    '^TNX',      # Ten-year Treasury Yield
]

start_date = dates['Statement Date'].min() - pd.Timedelta(days=15)
end_date = dates['Statement Date'].max() + pd.Timedelta(days=15)

all_indices_data = {}
for ticker in tickers:
    #print(f"Downloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data['return'] = data['Close'].pct_change()
    all_indices_data[ticker] = data[['return']].dropna()
    #print(f"Data for {ticker} downloaded.")

rows = []

for index, row_fomc in dates.iterrows():
    date = row_fomc['Statement Date']
    document_type = row_fomc['document_type']
    for ticker in tickers:
        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type} 
        for t in range(-15, 16):
            target_date = date + pd.Timedelta(days=t)
            if target_date in all_indices_data[ticker].index:
                row[f'T{t:+}'] = all_indices_data[ticker].loc[target_date, 'return']
            else:
                row[f'T{t:+}'] = pd.NA
        rows.append(row)

statements_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t:+}' for t in range(-15, 16)]
statements_df = statements_df[column_order]

statements_df

statements_df.to_csv('raw_data/statement_prices.csv', index=False)

/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_22308/128696647.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates['Statement Date'] = pd.to_datetime(dates['Statement Date'])


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
# First, ensure the announcement_date column is datetime (if it isn't already)
statements_df['announcement_date'] = pd.to_datetime(statements_df['announcement_date'])

# Now create a meeting_id based on unique announcement_date
statements_df['meeting_id'] = statements_df['announcement_date'].factorize()[0] + 1


In [8]:
statements_df

,announcement_date,ticker,document_type,T-15,T-14,T-13,T-12,T-11,T-10,T-9,...,T+7,T+8,T+9,T+10,T+11,T+12,T+13,T+14,T+15,meeting_id
0,2000-02-02,^GSPC,statement,<NA>,0.000522,-0.007095,-0.002912,<NA>,<NA>,-0.027634,...,-0.020815,0.003627,-0.020969,<NA>,<NA>,0.002033,0.008713,-0.010256,0.000425,1
1,2000-02-02,^IXIC,statement,<NA>,0.004958,0.009207,0.010954,<NA>,<NA>,-0.032894,...,-0.014514,0.02805,-0.020104,<NA>,<NA>,0.005255,0.000502,0.001556,0.027389,1
2,2000-02-02,^DJI,statement,<NA>,-0.006173,-0.012016,-0.008773,<NA>,<NA>,-0.021645,...,-0.023585,-0.00519,-0.020521,<NA>,<NA>,0.009077,0.018845,-0.014618,-0.004435,1
3,2000-02-02,^RUT,statement,<NA>,0.012776,0.013961,0.012631,<NA>,<NA>,-0.020583,...,-0.002772,0.011586,-0.009424,<NA>,<NA>,0.005288,0.000556,0.01392,0.019461,1
4,2000-02-02,^W5000,statement,<NA>,0.003503,-0.002435,0.001411,<NA>,<NA>,-0.026098,...,-0.018755,0.005698,-0.018061,<NA>,<NA>,0.002445,0.006289,-0.005964,0.004514,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,2025-03-19,XLB,statement,-0.013566,0.026107,-0.005225,0.00274,<NA>,<NA>,-0.020838,...,0.001849,-0.000461,-0.018352,<NA>,<NA>,0.010935,0.003722,0.008227,<NA>,200
3596,2025-03-19,XLK,statement,-0.000777,0.01405,-0.027757,0.014483,<NA>,<NA>,-0.042509,...,-0.022304,-0.009228,-0.024254,<NA>,<NA>,0.000484,0.007313,0.007404,<NA>,200
3597,2025-03-19,XLC,statement,-0.007922,0.012877,-0.017936,0.006422,<NA>,<NA>,-0.024928,...,-0.008505,-0.00444,-0.029498,<NA>,<NA>,0.007416,0.000622,0.003523,<NA>,200
3598,2025-03-19,^IRX,statement,0.000477,0.002384,-0.001902,0.0,<NA>,<NA>,-0.002859,...,0.001913,-0.000477,0.0,<NA>,<NA>,0.001194,0.001669,0.000714,<NA>,200


In [9]:
dates['Intermeeting Date'] = pd.to_datetime(dates['Intermeeting Date'])
dates['document_type'] = 'intermeeting' 

tickers = [
    '^GSPC',     # S&P 500
    '^IXIC',     # NASDAQ Composite
    '^DJI',      # Dow Jones Industrial Average
    '^RUT',      # Russell 2000
    '^W5000',    # Wilshire 5000
    'XLF',       # Financials Sector (ETF)
    'XLRE',      # Real Estate Sector (ETF)
    'XLU',       # Utilities Sector (ETF)
    'XLY',       # Consumer Discretionary Sector (ETF)
    'XLP',       # Consumer Staples Sector (ETF)
    'XLE',       # Energy Sector (ETF)
    'XLV',       # Healthcare Sector (ETF)
    'XLI',       # Industrials Sector (ETF)
    'XLB',       # Materials Sector (ETF)
    'XLK',       # Information Technology Sector (ETF)
    'XLC',       # Communication Services Sector (ETF)
    '^IRX',      # Three-month Treasury Bill Yield
    '^TNX',      # Ten-year Treasury Yield
]

start_date = dates['Intermeeting Date'].min() - pd.Timedelta(days=15)
end_date = dates['Intermeeting Date'].max() + pd.Timedelta(days=15)

all_indices_data = {}
for ticker in tickers:
    #print(f"Downloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data['return'] = data['Close'].pct_change()
    all_indices_data[ticker] = data[['return']].dropna()
    #print(f"Data for {ticker} downloaded.")

rows = []

for index, row_fomc in dates.iterrows():
    date = row_fomc['Intermeeting Date']
    document_type = row_fomc['document_type']
    for ticker in tickers:
        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type} 
        for t in range(-15, 16):
            target_date = date + pd.Timedelta(days=t)
            if target_date in all_indices_data[ticker].index:
                row[f'T{t:+}'] = all_indices_data[ticker].loc[target_date, 'return']
            else:
                row[f'T{t:+}'] = pd.NA
        rows.append(row)

intermeeting_df = pd.DataFrame(rows)


column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t:+}' for t in range(-15, 16)]
intermeeting_df = intermeeting_df[column_order]

intermeeting_df

intermeeting_df.to_csv('raw_data/intermeeting_prices.csv', index=False)

/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_22308/1459292757.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates['Intermeeting Date'] = pd.to_datetime(dates['Intermeeting Date'])
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*************

In [10]:
# First, ensure the announcement_date column is datetime (if it isn't already)
intermeeting_df['announcement_date'] = pd.to_datetime(intermeeting_df['announcement_date'])

# Now create a meeting_id based on unique announcement_date
intermeeting_df['meeting_id'] = intermeeting_df['announcement_date'].factorize()[0] + 1


In [11]:
intermeeting_df

,announcement_date,ticker,document_type,T-15,T-14,T-13,T-12,T-11,T-10,T-9,...,T+7,T+8,T+9,T+10,T+11,T+12,T+13,T+14,T+15,meeting_id
0,2000-03-23,^GSPC,intermeeting,<NA>,0.025602,-0.004723,<NA>,<NA>,-0.008207,-0.017685,...,-0.013656,0.007164,<NA>,<NA>,0.004931,-0.007464,-0.004924,0.009392,0.009998,1
1,2000-03-23,^IXIC,intermeeting,<NA>,0.030548,0.000349,<NA>,<NA>,-0.028004,-0.04088,...,-0.040214,0.025783,<NA>,<NA>,-0.076353,-0.017707,0.0049,0.023587,0.041919,1
2,2000-03-23,^DJI,intermeeting,<NA>,0.015644,-0.008182,<NA>,<NA>,0.001844,-0.013661,...,-0.003491,-0.005312,<NA>,<NA>,0.027469,-0.005087,-0.011726,0.007282,-0.000251,1
3,2000-03-23,^RUT,intermeeting,<NA>,0.01912,-0.003696,<NA>,<NA>,-0.02264,-0.029061,...,-0.02105,0.014147,<NA>,<NA>,-0.042757,-0.019223,0.023552,0.027913,0.0197,1
4,2000-03-23,^W5000,intermeeting,<NA>,0.024762,-0.00388,<NA>,<NA>,-0.01357,-0.021922,...,-0.01743,0.011479,<NA>,<NA>,-0.012209,-0.011746,-0.000518,0.014204,0.013746,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,2025-04-09,XLB,intermeeting,-0.000231,0.001849,-0.000461,-0.018352,<NA>,<NA>,0.010935,...,-0.007794,0.006858,<NA>,<NA>,<NA>,-0.015232,0.02339,0.001229,<NA>,200
3596,2025-04-09,XLK,intermeeting,0.003308,-0.022304,-0.009228,-0.024254,<NA>,<NA>,0.000484,...,-0.034745,-0.005415,<NA>,<NA>,<NA>,-0.025822,0.024377,0.028993,<NA>,200
3597,2025-04-09,XLC,intermeeting,0.012666,-0.008505,-0.00444,-0.029498,<NA>,<NA>,0.007416,...,-0.026924,0.006471,<NA>,<NA>,<NA>,-0.022503,0.026083,0.016026,<NA>,200
3598,2025-04-09,^IRX,intermeeting,0.0,0.001913,-0.000477,0.0,<NA>,<NA>,0.001194,...,0.001905,-0.000713,<NA>,<NA>,<NA>,0.001189,0.000713,-0.001187,<NA>,200


In [12]:
intermeeting_df.to_csv('raw_data/intermeeting_prices.csv', index=False)

In [13]:
statements_df

,announcement_date,ticker,document_type,T-15,T-14,T-13,T-12,T-11,T-10,T-9,...,T+7,T+8,T+9,T+10,T+11,T+12,T+13,T+14,T+15,meeting_id
0,2000-02-02,^GSPC,statement,<NA>,0.000522,-0.007095,-0.002912,<NA>,<NA>,-0.027634,...,-0.020815,0.003627,-0.020969,<NA>,<NA>,0.002033,0.008713,-0.010256,0.000425,1
1,2000-02-02,^IXIC,statement,<NA>,0.004958,0.009207,0.010954,<NA>,<NA>,-0.032894,...,-0.014514,0.02805,-0.020104,<NA>,<NA>,0.005255,0.000502,0.001556,0.027389,1
2,2000-02-02,^DJI,statement,<NA>,-0.006173,-0.012016,-0.008773,<NA>,<NA>,-0.021645,...,-0.023585,-0.00519,-0.020521,<NA>,<NA>,0.009077,0.018845,-0.014618,-0.004435,1
3,2000-02-02,^RUT,statement,<NA>,0.012776,0.013961,0.012631,<NA>,<NA>,-0.020583,...,-0.002772,0.011586,-0.009424,<NA>,<NA>,0.005288,0.000556,0.01392,0.019461,1
4,2000-02-02,^W5000,statement,<NA>,0.003503,-0.002435,0.001411,<NA>,<NA>,-0.026098,...,-0.018755,0.005698,-0.018061,<NA>,<NA>,0.002445,0.006289,-0.005964,0.004514,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,2025-03-19,XLB,statement,-0.013566,0.026107,-0.005225,0.00274,<NA>,<NA>,-0.020838,...,0.001849,-0.000461,-0.018352,<NA>,<NA>,0.010935,0.003722,0.008227,<NA>,200
3596,2025-03-19,XLK,statement,-0.000777,0.01405,-0.027757,0.014483,<NA>,<NA>,-0.042509,...,-0.022304,-0.009228,-0.024254,<NA>,<NA>,0.000484,0.007313,0.007404,<NA>,200
3597,2025-03-19,XLC,statement,-0.007922,0.012877,-0.017936,0.006422,<NA>,<NA>,-0.024928,...,-0.008505,-0.00444,-0.029498,<NA>,<NA>,0.007416,0.000622,0.003523,<NA>,200
3598,2025-03-19,^IRX,statement,0.000477,0.002384,-0.001902,0.0,<NA>,<NA>,-0.002859,...,0.001913,-0.000477,0.0,<NA>,<NA>,0.001194,0.001669,0.000714,<NA>,200


In [14]:
combined_df = pd.concat([statements_df, intermeeting_df], ignore_index=True)
combined_df = combined_df.dropna(subset=['announcement_date'])
combined_df = combined_df.sort_values(by='announcement_date').reset_index(drop=True)
combined_df['meeting_id'] = combined_df['announcement_date'].rank(method='dense').astype(int)
combined_df = combined_df[['meeting_id'] + [col for col in combined_df.columns if col != 'meeting_id']]
combined_df.to_csv('raw_data/combined_dates.csv', index = False)

In [15]:
statements_df

,announcement_date,ticker,document_type,T-15,T-14,T-13,T-12,T-11,T-10,T-9,...,T+7,T+8,T+9,T+10,T+11,T+12,T+13,T+14,T+15,meeting_id
0,2000-02-02,^GSPC,statement,<NA>,0.000522,-0.007095,-0.002912,<NA>,<NA>,-0.027634,...,-0.020815,0.003627,-0.020969,<NA>,<NA>,0.002033,0.008713,-0.010256,0.000425,1
1,2000-02-02,^IXIC,statement,<NA>,0.004958,0.009207,0.010954,<NA>,<NA>,-0.032894,...,-0.014514,0.02805,-0.020104,<NA>,<NA>,0.005255,0.000502,0.001556,0.027389,1
2,2000-02-02,^DJI,statement,<NA>,-0.006173,-0.012016,-0.008773,<NA>,<NA>,-0.021645,...,-0.023585,-0.00519,-0.020521,<NA>,<NA>,0.009077,0.018845,-0.014618,-0.004435,1
3,2000-02-02,^RUT,statement,<NA>,0.012776,0.013961,0.012631,<NA>,<NA>,-0.020583,...,-0.002772,0.011586,-0.009424,<NA>,<NA>,0.005288,0.000556,0.01392,0.019461,1
4,2000-02-02,^W5000,statement,<NA>,0.003503,-0.002435,0.001411,<NA>,<NA>,-0.026098,...,-0.018755,0.005698,-0.018061,<NA>,<NA>,0.002445,0.006289,-0.005964,0.004514,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,2025-03-19,XLB,statement,-0.013566,0.026107,-0.005225,0.00274,<NA>,<NA>,-0.020838,...,0.001849,-0.000461,-0.018352,<NA>,<NA>,0.010935,0.003722,0.008227,<NA>,200
3596,2025-03-19,XLK,statement,-0.000777,0.01405,-0.027757,0.014483,<NA>,<NA>,-0.042509,...,-0.022304,-0.009228,-0.024254,<NA>,<NA>,0.000484,0.007313,0.007404,<NA>,200
3597,2025-03-19,XLC,statement,-0.007922,0.012877,-0.017936,0.006422,<NA>,<NA>,-0.024928,...,-0.008505,-0.00444,-0.029498,<NA>,<NA>,0.007416,0.000622,0.003523,<NA>,200
3598,2025-03-19,^IRX,statement,0.000477,0.002384,-0.001902,0.0,<NA>,<NA>,-0.002859,...,0.001913,-0.000477,0.0,<NA>,<NA>,0.001194,0.001669,0.000714,<NA>,200


In [16]:
# Load Combined Returns Data
combined_returns = pd.read_csv('raw_data/combined_dates.csv')
print(combined_returns.columns.tolist())

t_columns = [f'T{t}' if t < 0 else f'T+{t}' if t > 0 else 'T0' for t in range(-10, 11)]

all_indices_data = {}

for ticker in combined_returns['ticker'].unique():
    ticker_data = combined_returns[combined_returns['ticker'] == ticker]
    ticker_data = ticker_data.set_index('announcement_date')  # Set date as index
    ticker_data.index = pd.to_datetime(ticker_data.index)  # Ensure datetime index
    all_indices_data[ticker] = ticker_data

print("All index data loaded.")

['meeting_id', 'announcement_date', 'ticker', 'document_type', 'T-15', 'T-14', 'T-13', 'T-12', 'T-11', 'T-10', 'T-9', 'T-8', 'T-7', 'T-6', 'T-5', 'T-4', 'T-3', 'T-2', 'T-1', 'T+0', 'T+1', 'T+2', 'T+3', 'T+4', 'T+5', 'T+6', 'T+7', 'T+8', 'T+9', 'T+10', 'T+11', 'T+12', 'T+13', 'T+14', 'T+15']
All index data loaded.


In [17]:
all_indices_data['^DJI']['T+0']

announcement_date
2000-02-02   -0.003428
2000-03-21    0.021264
2000-03-23    0.023297
2000-05-16    0.011731
2000-05-18    0.000700
                ...   
2025-01-08    0.002512
2025-01-29   -0.003051
2025-02-19    0.001599
2025-03-19    0.009219
2025-04-09    0.078704
Name: T+0, Length: 400, dtype: float64

In [18]:
combined_returns

,meeting_id,announcement_date,ticker,document_type,T-15,T-14,T-13,T-12,T-11,T-10,...,T+6,T+7,T+8,T+9,T+10,T+11,T+12,T+13,T+14,T+15
0,1,2000-02-02,^GSPC,statement,NaN,0.000522,-0.007095,-0.002912,NaN,NaN,...,0.012273,-0.020815,0.003627,-0.020969,NaN,NaN,0.002033,0.008713,-0.010256,0.000425
1,1,2000-02-02,^TNX,statement,NaN,-0.003408,0.003420,0.004890,NaN,NaN,...,-0.004826,0.010608,-0.001949,-0.005258,NaN,NaN,-0.012385,0.001988,-0.000763,0.003360
2,1,2000-02-02,^IRX,statement,NaN,0.022945,-0.009346,0.000000,NaN,NaN,...,0.009141,-0.005435,0.001822,-0.003636,NaN,NaN,-0.003650,0.021978,-0.005376,0.003604
3,1,2000-02-02,XLC,statement,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2000-02-02,XLK,statement,NaN,0.001764,0.005868,0.001167,NaN,NaN,...,0.009132,-0.012443,0.023768,-0.026573,NaN,NaN,0.000575,0.002872,-0.005155,0.022452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,400,2025-04-09,^DJI,intermeeting,0.000098,-0.003116,-0.003653,-0.016922,NaN,NaN,...,-0.003845,-0.017329,-0.013289,NaN,NaN,NaN,-0.024828,0.026632,0.010707,NaN
7196,400,2025-04-09,^IXIC,intermeeting,0.004578,-0.020405,-0.005307,-0.027019,NaN,NaN,...,-0.000494,-0.030673,-0.001270,NaN,NaN,NaN,-0.025515,0.027063,0.025007,NaN
7197,400,2025-04-09,^IRX,intermeeting,0.000000,0.001913,-0.000477,0.000000,NaN,NaN,...,0.001192,0.001905,-0.000713,NaN,NaN,NaN,0.001189,0.000713,-0.001187,NaN
7198,400,2025-04-09,XLU,intermeeting,-0.015991,0.006630,-0.000258,0.007363,NaN,NaN,...,-0.000129,-0.009011,0.010262,NaN,NaN,NaN,-0.023659,0.027262,0.004231,NaN


In [19]:
def get_trading_window(trading_dates, fed_date, returns_series, window=10):
    try:
        fed_idx = trading_dates.get_loc(fed_date)
    except KeyError:
        idx_array = trading_dates.get_indexer([fed_date], method='ffill')
        fed_idx = idx_array[0]
        if fed_idx == -1:
            return None

    if fed_idx < 0 or fed_idx >= len(returns_series):
        return None

    result = {}
    for t in range(-window, 0):
        if 0 <= fed_idx + t < len(returns_series):
            result[f'T{t:+}'] = returns_series.iloc[fed_idx + t]
        else:
            result[f'T{t:+}'] = pd.NA
    if 0 <= fed_idx < len(returns_series):
        result['T0'] = returns_series.iloc[fed_idx]
    else:
        result['T0'] = pd.NA
    for t in range(1, window + 1):
        if 0 <= fed_idx + t < len(returns_series):
            result[f'T{t:+}'] = returns_series.iloc[fed_idx + t]
        else:
            result[f'T{t:+}'] = pd.NA

    return result

In [20]:
rows = []

for idx, row_fomc in dates_statements.iterrows():
    date = row_fomc['Statement Date']
    document_type = row_fomc['document_type']

    for ticker in all_indices_data.keys():
        ticker_data = all_indices_data[ticker]

        if date not in ticker_data.index:
            continue  # skip if no data

        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type}

        available_returns = ticker_data.loc[date]

        pre_event_returns = []
        for t in range(-15, 0):
            col_name = f'T{t}'
            if col_name in available_returns and pd.notna(available_returns[col_name]):
                pre_event_returns.append((t, available_returns[col_name]))
            if len(pre_event_returns) == 10:
                break

        post_event_returns = []
        for t in range(1, 16):
            col_name = f'T+{t}'
            if col_name in available_returns and pd.notna(available_returns[col_name]):
                post_event_returns.append((t, available_returns[col_name]))
            if len(post_event_returns) == 10:
                break

        t0_value = available_returns.get('T+0', pd.NA)

        for i in range(-10, 0):
            if len(pre_event_returns) >= abs(i):
                row[f'T{i}'] = pre_event_returns[i + 10 - 1][1]  # -10 is idx 0
            else:
                row[f'T{i}'] = pd.NA

        row['T0'] = t0_value

        for i in range(1, 11):
            if len(post_event_returns) >= i:
                row[f'T+{i}'] = post_event_returns[i - 1][1]
            else:
                row[f'T+{i}'] = pd.NA

        rows.append(row)

statements_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t}' if t < 0 else f'T+{t}' if t > 0 else 'T0' for t in range(-10, 11)]
for col in column_order:
    if col not in statements_df.columns:
        statements_df[col] = pd.NA
statements_df = statements_df[column_order]

print("Statement prices processed.")

Statement prices processed.


In [21]:
rows = []

for idx, row_fomc in dates_intermeetings.iterrows():
    date = row_fomc['Intermeeting Date']
    document_type = row_fomc['document_type']

    for ticker in all_indices_data.keys():
        ticker_data = all_indices_data[ticker]

        if date not in ticker_data.index:
            continue  

        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type}

        available_returns = ticker_data.loc[date]

        pre_event_returns = []
        for t in range(-15, 0):
            col_name = f'T{t}'
            if col_name in available_returns.index and pd.notna(available_returns[col_name]):
                pre_event_returns.append((t, available_returns[col_name]))
            if len(pre_event_returns) == 10:
                break

        post_event_returns = []
        for t in range(1, 16):
            col_name = f'T+{t}'
            if col_name in available_returns.index and pd.notna(available_returns[col_name]):
                post_event_returns.append((t, available_returns[col_name]))
            if len(post_event_returns) == 10:
                break

        t0_value = available_returns.get('T+0', pd.NA)

        for i in range(-10, 0):
            idx = abs(i) - 1
            if idx < len(pre_event_returns):
                row[f'T{i}'] = pre_event_returns[idx][1]
            else:
                row[f'T{i}'] = pd.NA
        
        row['T0'] = t0_value
        
        for i in range(1, 11):
            idx = i - 1
            if idx < len(post_event_returns):
                row[f'T+{i}'] = post_event_returns[idx][1]
            else:
                row[f'T+{i}'] = pd.NA


        rows.append(row)

intermeeting_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t}' if t < 0 else f'T+{t}' if t > 0 else 'T0' for t in range(-10, 11)]
for col in column_order:
    if col not in intermeeting_df.columns:
        intermeeting_df[col] = pd.NA
intermeeting_df = intermeeting_df[column_order]

print("Intermeeting prices processed.")

Intermeeting prices processed.


In [22]:
combined_df = pd.concat([statements_df, intermeeting_df], ignore_index=True)
combined_df

,announcement_date,ticker,document_type,T-10,T-9,T-8,T-7,T-6,T-5,T-4,...,T+1,T+2,T+3,T+4,T+5,T+6,T+7,T+8,T+9,T+10
0,2000-02-02,^GSPC,statement,0.010628,0.000522,-0.007095,-0.002912,-0.027634,0.006065,-0.004213,...,0.011248,-0.000421,-0.000091,0.012273,-0.020815,0.003627,-0.020969,0.002033,0.008713,-0.010256
1,2000-02-02,^TNX,statement,-0.0075,-0.003408,0.00342,0.00489,-0.014305,0.0,-0.000598,...,-0.018642,0.006487,0.017493,-0.004826,0.010608,-0.001949,-0.005258,-0.012385,0.001988,-0.000763
2,2000-02-02,^IRX,statement,0.003617,0.022945,-0.009346,0.0,0.00566,0.013133,0.001852,...,-0.012727,0.01105,-0.003643,0.009141,-0.005435,0.001822,-0.003636,-0.00365,0.021978,-0.005376
3,2000-02-02,XLC,statement,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,2000-02-02,XLK,statement,0.021014,0.001764,0.005868,0.001167,-0.032634,0.021386,-0.032734,...,0.028846,0.011682,0.011547,0.009132,-0.012443,0.023768,-0.026573,0.000575,0.002872,-0.005155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,2025-04-09,^W5000,intermeeting,-0.002977,-0.05874,-0.05105,0.00802,0.003932,0.004373,-0.020066,...,-0.035874,0.017714,0.008244,-0.001298,-0.021046,0.002311,-0.023624,0.025441,0.016976,<NA>
7196,2025-04-09,^RUT,intermeeting,-0.009245,-0.043715,-0.06591,0.016459,0.000164,-0.005615,-0.02054,...,-0.042741,0.015731,0.011117,0.001085,-0.010324,0.009198,-0.021429,0.027147,0.015268,<NA>
7197,2025-04-09,^DJI,intermeeting,-0.009115,-0.055026,-0.039772,0.005605,-0.000281,0.010049,-0.016922,...,-0.02499,0.015635,0.007761,-0.003845,-0.017329,-0.013289,-0.024828,0.026632,0.010707,<NA>
7198,2025-04-09,^IXIC,intermeeting,0.000992,-0.058174,-0.059681,0.008663,0.008706,-0.001368,-0.027019,...,-0.043075,0.020574,0.006399,-0.000494,-0.030673,-0.00127,-0.025515,0.027063,0.025007,<NA>


In [23]:
combined_df = pd.concat([statements_df, intermeeting_df], ignore_index=True)
combined_df = combined_df.dropna(subset=['announcement_date'])
combined_df = combined_df.sort_values(by='announcement_date').reset_index(drop=True)
combined_df['meeting_id'] = combined_df['announcement_date'].rank(method='dense').astype(int)
combined_df = combined_df[['meeting_id'] + [col for col in combined_df.columns if col != 'meeting_id']]
combined_by_date = combined_df
combined_by_date

,meeting_id,announcement_date,ticker,document_type,T-10,T-9,T-8,T-7,T-6,T-5,...,T+1,T+2,T+3,T+4,T+5,T+6,T+7,T+8,T+9,T+10
0,1,2000-02-02,^GSPC,statement,0.010628,0.000522,-0.007095,-0.002912,-0.027634,0.006065,...,0.011248,-0.000421,-0.000091,0.012273,-0.020815,0.003627,-0.020969,0.002033,0.008713,-0.010256
1,1,2000-02-02,XLY,statement,0.036185,-0.002086,-0.013584,-0.020656,-0.037318,0.00618,...,0.003906,-0.001668,-0.021158,0.022753,-0.016686,-0.009049,-0.01427,0.005212,0.002304,-0.03908
2,1,2000-02-02,^IXIC,statement,0.02833,0.004958,0.009207,0.010954,-0.032894,0.017414,...,0.033633,0.007875,0.018291,0.024465,-0.014514,0.02805,-0.020104,0.005255,0.000502,0.001556
3,1,2000-02-02,^DJI,statement,0.009188,-0.006173,-0.012016,-0.008773,-0.021645,0.001973,...,0.000931,-0.004507,-0.005291,0.004751,-0.023585,-0.00519,-0.020521,0.009077,0.018845,-0.014618
4,1,2000-02-02,^RUT,statement,0.015154,0.012776,0.013961,0.012631,-0.020583,-0.002601,...,0.023025,0.007457,0.013073,0.009579,-0.002772,0.011586,-0.009424,0.005288,0.000556,0.01392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,400,2025-04-09,^IRX,intermeeting,-0.001203,-0.007168,-0.004283,0.000714,0.001669,0.001194,...,-0.007332,0.003812,-0.004272,0.001192,0.001905,-0.000713,0.001189,0.000713,-0.001187,<NA>
7196,400,2025-04-09,^TNX,intermeeting,0.04266,-0.017263,-0.033603,0.009625,-0.021196,-0.002115,...,-0.001364,0.022531,-0.028711,-0.009395,-0.010178,0.01262,0.016617,-0.003632,-0.000456,<NA>
7197,400,2025-04-09,^IXIC,intermeeting,0.000992,-0.058174,-0.059681,0.008663,0.008706,-0.001368,...,-0.043075,0.020574,0.006399,-0.000494,-0.030673,-0.00127,-0.025515,0.027063,0.025007,<NA>
7198,400,2025-04-09,XLE,intermeeting,-0.006475,-0.091999,-0.078508,0.001383,0.005886,0.011035,...,-0.065323,0.024812,0.003549,-0.001516,0.008223,0.022585,-0.026135,0.02545,-0.001843,<NA>


In [24]:
combined_by_date.to_csv('raw_data/combined_by_date.csv', index = False)

In [25]:
combined_df = combined_df.copy()

combined_df = combined_df.sort_values(by=['document_type', 'announcement_date'])

combined_df['meeting_id'] = (
    combined_df.groupby('document_type')['announcement_date']
    .transform(lambda x: pd.factorize(x)[0] + 1)  
)

combined_by_id = combined_df.sort_values(by='announcement_date').reset_index(drop=True)
combined_by_id

,meeting_id,announcement_date,ticker,document_type,T-10,T-9,T-8,T-7,T-6,T-5,...,T+1,T+2,T+3,T+4,T+5,T+6,T+7,T+8,T+9,T+10
0,1,2000-02-02,XLP,statement,-0.010087,0.007237,-0.013716,-0.011259,-0.028801,-0.012413,...,-0.01356,-0.000687,-0.001375,0.012396,-0.008843,-0.033631,-0.019176,0.005069,0.022334,-0.014094
1,1,2000-02-02,^TNX,statement,-0.0075,-0.003408,0.00342,0.00489,-0.014305,0.0,...,-0.018642,0.006487,0.017493,-0.004826,0.010608,-0.001949,-0.005258,-0.012385,0.001988,-0.000763
2,1,2000-02-02,^IRX,statement,0.003617,0.022945,-0.009346,0.0,0.00566,0.013133,...,-0.012727,0.01105,-0.003643,0.009141,-0.005435,0.001822,-0.003636,-0.00365,0.021978,-0.005376
3,1,2000-02-02,XLC,statement,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,1,2000-02-02,XLK,statement,0.021014,0.001764,0.005868,0.001167,-0.032634,0.021386,...,0.028846,0.011682,0.011547,0.009132,-0.012443,0.023768,-0.026573,0.000575,0.002872,-0.005155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,200,2025-04-09,^IRX,intermeeting,-0.001203,-0.007168,-0.004283,0.000714,0.001669,0.001194,...,-0.007332,0.003812,-0.004272,0.001192,0.001905,-0.000713,0.001189,0.000713,-0.001187,<NA>
7196,200,2025-04-09,^TNX,intermeeting,0.04266,-0.017263,-0.033603,0.009625,-0.021196,-0.002115,...,-0.001364,0.022531,-0.028711,-0.009395,-0.010178,0.01262,0.016617,-0.003632,-0.000456,<NA>
7197,200,2025-04-09,^IXIC,intermeeting,0.000992,-0.058174,-0.059681,0.008663,0.008706,-0.001368,...,-0.043075,0.020574,0.006399,-0.000494,-0.030673,-0.00127,-0.025515,0.027063,0.025007,<NA>
7198,200,2025-04-09,^GSPC,intermeeting,-0.002331,-0.05975,-0.048396,0.006728,0.003781,0.005539,...,-0.034608,0.018092,0.007945,-0.001728,-0.022408,0.001327,-0.023567,0.025117,0.016661,<NA>


In [26]:
combined_by_id .to_csv('raw_data/combined_by_id.csv', index = False)

# Sentiment Analysis

In [27]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup
import fnmatch
import glob
import os
import re
from time import sleep
from zipfile import ZipFile

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from utils.near_regex import * 
from tqdm import tqdm  


with open('inputs/ML_positive_unigram.txt', 'r') as file:
    BHR_positive = sorted([line.strip().lower() for line in file])

with open('inputs/ML_negative_unigram.txt', 'r') as file:
    BHR_negative = sorted([line.strip().lower() for line in file])

lm_df = pd.read_csv("inputs/LM_MasterDictionary_1993-2021.csv")
LM_positive = sorted([w.lower() for w in lm_df[lm_df['Positive'] > 0]['Word'].tolist()])
LM_negative = sorted([w.lower() for w in lm_df[lm_df['Negative'] > 0]['Word'].tolist()])

# Compile regex
BHR_positive_regex = re.compile(r'\b(?:' + '|'.join(map(re.escape, BHR_positive)) + r')\b')
BHR_negative_regex = re.compile(r'\b(?:' + '|'.join(map(re.escape, BHR_negative)) + r')\b')
LM_positive_regex = re.compile(r'\b(?:' + '|'.join(map(re.escape, LM_positive)) + r')\b')
LM_negative_regex = re.compile(r'\b(?:' + '|'.join(map(re.escape, LM_negative)) + r')\b')


In [28]:
def read_file_with_fallback(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    except UnicodeDecodeError:
        with open(filepath, 'r', encoding='ISO-8859-1') as f:
            return f.read()

def compute_sentiment_scores(folder):
    scores = []

    for filename in os.listdir(folder):
        if filename.endswith('.html'):
            filepath = os.path.join(folder, filename)
            html_content = read_file_with_fallback(filepath)
            soup = BeautifulSoup(html_content, 'html.parser')
            text = soup.get_text(separator=' ', strip=True).lower()
            words = text.split()
            total_words = len(words)

            bhr_pos = len(BHR_positive_regex.findall(text)) / total_words if total_words else 0
            bhr_neg = len(BHR_negative_regex.findall(text)) / total_words if total_words else 0
            lm_pos = len(LM_positive_regex.findall(text)) / total_words if total_words else 0
            lm_neg = len(LM_negative_regex.findall(text)) / total_words if total_words else 0

            scores.append({
                'filename': filename,
                'BHR_Positive_Score': bhr_pos,
                'BHR_Negative_Score': bhr_neg,
                'LM_Positive_Score': lm_pos,
                'LM_Negative_Score': lm_neg
            })

    return pd.DataFrame(scores)

announcements_df = compute_sentiment_scores('raw_data/announcements')
intermeeting_df = compute_sentiment_scores('raw_data/intermeeting')

In [29]:
announcements_df

,filename,BHR_Positive_Score,BHR_Negative_Score,LM_Positive_Score,LM_Negative_Score
0,monetary20170920a.html,0.010702,0.021403,0.011296,0.008323
1,monetary20120913a.html,0.015892,0.015892,0.015892,0.010006
2,monetary20200315a.html,0.013043,0.019022,0.013043,0.008152
3,monetary20011211a.html,0.030837,0.044053,0.017621,0.013216
4,monetary20221102a.html,0.009132,0.013046,0.011089,0.009132
...,...,...,...,...,...
195,monetary20090624a.html,0.011921,0.011258,0.013907,0.007947
196,monetary20030506a.html,0.040816,0.020408,0.024490,0.012245
197,monetary20090812a.html,0.012887,0.010954,0.014175,0.007732
198,monetary20020130a.html,0.037500,0.031250,0.018750,0.025000


In [30]:
intermeeting_df

,filename,BHR_Positive_Score,BHR_Negative_Score,LM_Positive_Score,LM_Negative_Score
0,fomcminutes20130501.html,0.029708,0.032226,0.011455,0.017120
1,fomcminutes20050920.html,0.031686,0.030233,0.012791,0.017151
2,fomcminutes20230503.html,0.023652,0.026862,0.007491,0.023009
3,fomcminutes20160427.html,0.028439,0.028439,0.012436,0.018992
4,fomcminutes20081029.html,0.023055,0.031575,0.004260,0.028944
...,...,...,...,...,...
195,fomcminutes20040630.html,0.037690,0.025788,0.014988,0.014327
196,fomcminutes20050630.html,0.032400,0.032863,0.012960,0.016431
197,fomcminutes20020924.html,0.034492,0.025540,0.016324,0.023697
198,fomcminutes20001003.html,0.035241,0.033916,0.010864,0.018283


In [31]:
announcements_df = announcements_df.copy()
intermeeting_df = intermeeting_df.copy()

announcements_df['meeting_id'] = range(1, len(announcements_df) + 1)
intermeeting_df['meeting_id'] = range(1, len(intermeeting_df) + 1)

announcements_df['doc_type'] = 'statement'
intermeeting_df['doc_type'] = 'intermeeting'

In [32]:
sentiment_df = pd.concat([announcements_df, intermeeting_df], ignore_index=True)
sentiment_df

,filename,BHR_Positive_Score,BHR_Negative_Score,LM_Positive_Score,LM_Negative_Score,meeting_id,doc_type
0,monetary20170920a.html,0.010702,0.021403,0.011296,0.008323,1,statement
1,monetary20120913a.html,0.015892,0.015892,0.015892,0.010006,2,statement
2,monetary20200315a.html,0.013043,0.019022,0.013043,0.008152,3,statement
3,monetary20011211a.html,0.030837,0.044053,0.017621,0.013216,4,statement
4,monetary20221102a.html,0.009132,0.013046,0.011089,0.009132,5,statement
...,...,...,...,...,...,...,...
395,fomcminutes20040630.html,0.037690,0.025788,0.014988,0.014327,196,intermeeting
396,fomcminutes20050630.html,0.032400,0.032863,0.012960,0.016431,197,intermeeting
397,fomcminutes20020924.html,0.034492,0.025540,0.016324,0.023697,198,intermeeting
398,fomcminutes20001003.html,0.035241,0.033916,0.010864,0.018283,199,intermeeting


In [33]:
combined_with_sentiment = combined_by_id.merge(
    sentiment_df,
    how='left',
    left_on=['meeting_id', 'document_type'],
    right_on=['meeting_id', 'doc_type']
)
combined_with_sentiment.drop(columns=['doc_type'], inplace=True)
combined_with_sentiment.drop(columns=['filename'], inplace=True)
combined_with_sentiment

,meeting_id,announcement_date,ticker,document_type,T-10,T-9,T-8,T-7,T-6,T-5,...,T+5,T+6,T+7,T+8,T+9,T+10,BHR_Positive_Score,BHR_Negative_Score,LM_Positive_Score,LM_Negative_Score
0,1,2000-02-02,XLP,statement,-0.010087,0.007237,-0.013716,-0.011259,-0.028801,-0.012413,...,-0.008843,-0.033631,-0.019176,0.005069,0.022334,-0.014094,0.010702,0.021403,0.011296,0.008323
1,1,2000-02-02,^TNX,statement,-0.0075,-0.003408,0.00342,0.00489,-0.014305,0.0,...,0.010608,-0.001949,-0.005258,-0.012385,0.001988,-0.000763,0.010702,0.021403,0.011296,0.008323
2,1,2000-02-02,^IRX,statement,0.003617,0.022945,-0.009346,0.0,0.00566,0.013133,...,-0.005435,0.001822,-0.003636,-0.00365,0.021978,-0.005376,0.010702,0.021403,0.011296,0.008323
3,1,2000-02-02,XLC,statement,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.010702,0.021403,0.011296,0.008323
4,1,2000-02-02,XLK,statement,0.021014,0.001764,0.005868,0.001167,-0.032634,0.021386,...,-0.012443,0.023768,-0.026573,0.000575,0.002872,-0.005155,0.010702,0.021403,0.011296,0.008323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,200,2025-04-09,^IRX,intermeeting,-0.001203,-0.007168,-0.004283,0.000714,0.001669,0.001194,...,0.001905,-0.000713,0.001189,0.000713,-0.001187,<NA>,0.038684,0.028724,0.012740,0.012972
7196,200,2025-04-09,^TNX,intermeeting,0.04266,-0.017263,-0.033603,0.009625,-0.021196,-0.002115,...,-0.010178,0.01262,0.016617,-0.003632,-0.000456,<NA>,0.038684,0.028724,0.012740,0.012972
7197,200,2025-04-09,^IXIC,intermeeting,0.000992,-0.058174,-0.059681,0.008663,0.008706,-0.001368,...,-0.030673,-0.00127,-0.025515,0.027063,0.025007,<NA>,0.038684,0.028724,0.012740,0.012972
7198,200,2025-04-09,^GSPC,intermeeting,-0.002331,-0.05975,-0.048396,0.006728,0.003781,0.005539,...,-0.022408,0.001327,-0.023567,0.025117,0.016661,<NA>,0.038684,0.028724,0.012740,0.012972


In [34]:
combined_with_sentiment.to_csv('raw_data/combined_with_sentiment.csv')

#### Issue
Still have some issues with the htmls - there is a lot of nonsense in some of the statements/intermeeting that is not 
actually part of the statement/intermeeting so we need to parse the document and get rid of that stuff 

This is going to be tedious there are many different formats of the htmls that have changed over the past 20 years.
Similar to how we had to change the exact URL when scraping

### Contextual Sentiment Analysis by topic 

In [35]:
import importlib
import utils.near_regex

importlib.reload(utils.near_regex)


<module 'utils.near_regex' from '/Users/brianfullenbaum/Desktop/FIN 377/Fed_Announcements/utils/near_regex.py'>

In [36]:
NEAR_finder(["interest rate"], ["increase"], "The Federal Reserve decided to increase the interest rate today.", max_words_between=5)


(1, ['increase the interest rate'])

In [37]:
import os
import re
from bs4 import BeautifulSoup
import pandas as pd



mpd_keywords = [
    "federal funds rate", "policy rate", "interest rate", "monetary policy stance",
    "tightening", "easing", "accommodative policy", "restrictive policy", "basis points"
]

folders = {
    'statement': 'raw_data/announcements/',
    'intermeeting': 'raw_data/intermeeting/'
}

results = []

for doc_type, folder_path in folders.items():
    for filename in os.listdir(folder_path):
        if filename.endswith('.html'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='latin1') as f:
                html_text = re.sub(r'\s+', ' ', BeautifulSoup(f, 'html.parser').get_text(" ").lower())
                word_count = len(html_text.split())

                pos_matches, _ = NEAR_finder(mpd_keywords, BHR_positive, html_text, max_words_between=5, greedy=False)
                neg_matches, _ = NEAR_finder(mpd_keywords, BHR_negative, html_text, max_words_between=5, greedy=False)

                pos_score = pos_matches / word_count if word_count else 0
                neg_score = neg_matches / word_count if word_count else 0

                results.append({
                    'filename': filename,
                    'doc_type': doc_type,
                    'MP_Positive_Score': pos_score,
                    'MP_Negative_Score': neg_score
                })

# Final result
monetary_policy_sentiment_df = pd.DataFrame(results)
monetary_policy_sentiment_df

,filename,doc_type,MP_Positive_Score,MP_Negative_Score
0,monetary20170920a.html,statement,0.000000,0.000000
1,monetary20120913a.html,statement,0.000000,0.000000
2,monetary20200315a.html,statement,0.000000,0.000543
3,monetary20011211a.html,statement,0.004405,0.008811
4,monetary20221102a.html,statement,0.000000,0.000000
...,...,...,...,...
395,fomcminutes20040630.html,intermeeting,0.000441,0.000882
396,fomcminutes20050630.html,intermeeting,0.001157,0.000694
397,fomcminutes20020924.html,intermeeting,0.000000,0.000000
398,fomcminutes20001003.html,intermeeting,0.000000,0.000530


In [38]:

econ_keywords = [
    "inflation", "employment", "unemployment", "economic activity", "growth",
    "GDP", "consumer spending", "business investment", "labor market",
    "price stability", "economic outlook"
]

folders = {
    'statement': 'raw_data/announcements/',
    'intermeeting': 'raw_data/intermeeting/'
}

econ_results = []

for doc_type, folder_path in folders.items():
    for filename in os.listdir(folder_path):
        if filename.endswith('.html'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='latin1') as f:
                html_text = re.sub(r'\s+', ' ', BeautifulSoup(f, 'html.parser').get_text(" ").lower())
                word_count = len(html_text.split())

                pos_matches, _ = NEAR_finder(econ_keywords, BHR_positive, html_text, max_words_between=5, greedy=False)
                neg_matches, _ = NEAR_finder(econ_keywords, BHR_negative, html_text, max_words_between=5, greedy=False)

                pos_score = pos_matches / word_count if word_count else 0
                neg_score = neg_matches / word_count if word_count else 0

                econ_results.append({
                    'filename': filename,
                    'doc_type': doc_type,
                    'EC_Positive_Score': pos_score,
                    'EC_Negative_Score': neg_score
                })

economic_conditions_sentiment_df = pd.DataFrame(econ_results)
economic_conditions_sentiment_df


,filename,doc_type,EC_Positive_Score,EC_Negative_Score
0,monetary20170920a.html,statement,0.001784,0.005351
1,monetary20120913a.html,statement,0.005297,0.002354
2,monetary20200315a.html,statement,0.002174,0.003261
3,monetary20011211a.html,statement,0.008811,0.013216
4,monetary20221102a.html,statement,0.001305,0.001957
...,...,...,...,...
395,fomcminutes20040630.html,intermeeting,0.008596,0.005069
396,fomcminutes20050630.html,intermeeting,0.008563,0.006711
397,fomcminutes20020924.html,intermeeting,0.006056,0.004739
398,fomcminutes20001003.html,intermeeting,0.006889,0.007419


In [39]:

fg_keywords = [
    "future", "coming months", "coming meetings", "outlook", "path",
    "trajectory", "forward guidance", "anticipate", "expect", "foresee", "project"
]

folders = {
    'statement': 'raw_data/announcements/',
    'intermeeting': 'raw_data/intermeeting/'
}

fg_results = []

for doc_type, folder_path in folders.items():
    for filename in os.listdir(folder_path):
        if filename.endswith('.html'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='latin1') as f:
                html_text = re.sub(r'\s+', ' ', BeautifulSoup(f, 'html.parser').get_text(" ").lower())
                word_count = len(html_text.split())

                pos_matches, _ = NEAR_finder(fg_keywords, BHR_positive, html_text, max_words_between=5, greedy=False)
                neg_matches, _ = NEAR_finder(fg_keywords, BHR_negative, html_text, max_words_between=5, greedy=False)

                pos_score = pos_matches / word_count if word_count else 0
                neg_score = neg_matches / word_count if word_count else 0

                fg_results.append({
                    'filename': filename,
                    'doc_type': doc_type,
                    'FG_Positive_Score': pos_score,
                    'FG_Negative_Score': neg_score
                })

forward_guidance_sentiment_df = pd.DataFrame(fg_results)
forward_guidance_sentiment_df


,filename,doc_type,FG_Positive_Score,FG_Negative_Score
0,monetary20170920a.html,statement,0.000595,0.001784
1,monetary20120913a.html,statement,0.000000,0.000589
2,monetary20200315a.html,statement,0.000543,0.000543
3,monetary20011211a.html,statement,0.000000,0.004405
4,monetary20221102a.html,statement,0.000000,0.000000
...,...,...,...,...
395,fomcminutes20040630.html,intermeeting,0.000661,0.000441
396,fomcminutes20050630.html,intermeeting,0.000463,0.000463
397,fomcminutes20020924.html,intermeeting,0.001843,0.001316
398,fomcminutes20001003.html,intermeeting,0.000000,0.000795


In [40]:
import os
import re
from bs4 import BeautifulSoup
import pandas as pd


bsp_keywords = [
    "balance sheet", "asset purchase", "securities", "treasury securities", "agency debt",
    "mortgage-backed securities", "mbs", "portfolio", "holdings", "reinvestment",
    "quantitative easing", "qe", "runoff"
]

folders = {
    'statement': 'raw_data/announcements/',
    'intermeeting': 'raw_data/intermeeting/'
}

bsp_results = []

for doc_type, folder_path in folders.items():
    for filename in os.listdir(folder_path):
        if filename.endswith('.html'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='latin1') as f:
                html_text = re.sub(r'\s+', ' ', BeautifulSoup(f, 'html.parser').get_text(" ").lower())
                word_count = len(html_text.split())

                pos_matches, _ = NEAR_finder(bsp_keywords, BHR_positive, html_text, max_words_between=5, greedy=False)
                neg_matches, _ = NEAR_finder(bsp_keywords, BHR_negative, html_text, max_words_between=5, greedy=False)

                pos_score = pos_matches / word_count if word_count else 0
                neg_score = neg_matches / word_count if word_count else 0

                bsp_results.append({
                    'filename': filename,
                    'doc_type': doc_type,
                    'BSP_Positive_Score': pos_score,
                    'BSP_Negative_Score': neg_score
                })

balance_sheet_policy_sentiment_df = pd.DataFrame(bsp_results)
balance_sheet_policy_sentiment_df


,filename,doc_type,BSP_Positive_Score,BSP_Negative_Score
0,monetary20170920a.html,statement,0.000000,0.000000
1,monetary20120913a.html,statement,0.001177,0.001766
2,monetary20200315a.html,statement,0.001087,0.000000
3,monetary20011211a.html,statement,0.000000,0.000000
4,monetary20221102a.html,statement,0.000652,0.000000
...,...,...,...,...
395,fomcminutes20040630.html,intermeeting,0.000882,0.000220
396,fomcminutes20050630.html,intermeeting,0.000231,0.000463
397,fomcminutes20020924.html,intermeeting,0.000000,0.000000
398,fomcminutes20001003.html,intermeeting,0.000000,0.000265


In [41]:
def assign_meeting_ids(df):
    df = df.copy()
    df['meeting_id'] = 0
    for doc_type in ['statement', 'intermeeting']:
        mask = df['doc_type'] == doc_type
        df.loc[mask, 'meeting_id'] = range(1, mask.sum() + 1)
    return df

monetary_policy_sentiment_df = assign_meeting_ids(monetary_policy_sentiment_df)
economic_conditions_sentiment_df = assign_meeting_ids(economic_conditions_sentiment_df)
forward_guidance_sentiment_df = assign_meeting_ids(forward_guidance_sentiment_df)
balance_sheet_policy_sentiment_df = assign_meeting_ids(balance_sheet_policy_sentiment_df)

In [42]:


sentiment_all_topics = monetary_policy_sentiment_df[['meeting_id', 'doc_type', 'MP_Positive_Score', 'MP_Negative_Score']] \
    .merge(economic_conditions_sentiment_df[['meeting_id', 'doc_type', 'EC_Positive_Score', 'EC_Negative_Score']],
           on=['meeting_id', 'doc_type'], how='outer') \
    .merge(forward_guidance_sentiment_df[['meeting_id', 'doc_type', 'FG_Positive_Score', 'FG_Negative_Score']],
           on=['meeting_id', 'doc_type'], how='outer') \
    .merge(balance_sheet_policy_sentiment_df[['meeting_id', 'doc_type', 'BSP_Positive_Score', 'BSP_Negative_Score']],
           on=['meeting_id', 'doc_type'], how='outer')

In [43]:
new_combined_with_sentiment = combined_with_sentiment.merge(
    sentiment_all_topics,
    left_on=['meeting_id', 'document_type'],
    right_on=['meeting_id', 'doc_type'],
    how='left'
).drop(columns=['doc_type'])  

In [44]:
new_combined_with_sentiment


,meeting_id,announcement_date,ticker,document_type,T-10,T-9,T-8,T-7,T-6,T-5,...,LM_Positive_Score,LM_Negative_Score,MP_Positive_Score,MP_Negative_Score,EC_Positive_Score,EC_Negative_Score,FG_Positive_Score,FG_Negative_Score,BSP_Positive_Score,BSP_Negative_Score
0,1,2000-02-02,XLP,statement,-0.010087,0.007237,-0.013716,-0.011259,-0.028801,-0.012413,...,0.011296,0.008323,0.000000,0.000000,0.001784,0.005351,0.000595,0.001784,0.000000,0.000000
1,1,2000-02-02,^TNX,statement,-0.0075,-0.003408,0.00342,0.00489,-0.014305,0.0,...,0.011296,0.008323,0.000000,0.000000,0.001784,0.005351,0.000595,0.001784,0.000000,0.000000
2,1,2000-02-02,^IRX,statement,0.003617,0.022945,-0.009346,0.0,0.00566,0.013133,...,0.011296,0.008323,0.000000,0.000000,0.001784,0.005351,0.000595,0.001784,0.000000,0.000000
3,1,2000-02-02,XLC,statement,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.011296,0.008323,0.000000,0.000000,0.001784,0.005351,0.000595,0.001784,0.000000,0.000000
4,1,2000-02-02,XLK,statement,0.021014,0.001764,0.005868,0.001167,-0.032634,0.021386,...,0.011296,0.008323,0.000000,0.000000,0.001784,0.005351,0.000595,0.001784,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,200,2025-04-09,^IRX,intermeeting,-0.001203,-0.007168,-0.004283,0.000714,0.001669,0.001194,...,0.012740,0.012972,0.001158,0.000463,0.007181,0.006949,0.000927,0.000927,0.000463,0.000232
7196,200,2025-04-09,^TNX,intermeeting,0.04266,-0.017263,-0.033603,0.009625,-0.021196,-0.002115,...,0.012740,0.012972,0.001158,0.000463,0.007181,0.006949,0.000927,0.000927,0.000463,0.000232
7197,200,2025-04-09,^IXIC,intermeeting,0.000992,-0.058174,-0.059681,0.008663,0.008706,-0.001368,...,0.012740,0.012972,0.001158,0.000463,0.007181,0.006949,0.000927,0.000927,0.000463,0.000232
7198,200,2025-04-09,^GSPC,intermeeting,-0.002331,-0.05975,-0.048396,0.006728,0.003781,0.005539,...,0.012740,0.012972,0.001158,0.000463,0.007181,0.006949,0.000927,0.000927,0.000463,0.000232


In [45]:
new_combined_with_sentiment.to_csv('raw_data/new_combined_with_sentiment.csv', index = False)

#### Problems still 
The word lists need to be more extensive because there are some 0 scores in the outputs 
for the contextual sentiments for some of the annoucnements

Also still need to get rid of nonsense text that is in some of the documents

### ChatGPT Use Case Addition

pip install openai

In [5]:
import os
import re
from bs4 import BeautifulSoup
from datetime import datetime
from openai import OpenAI



In [6]:
folder_path = "raw_data/announcements"
html_files = [f for f in os.listdir(folder_path) if f.endswith(".html")]

def extract_date_from_filename(filename):
    match = re.search(r"monetary(\d{8})", filename)
    if match:
        return datetime.strptime(match.group(1), "%Y%m%d")
    return None

dated_files = [(extract_date_from_filename(f), f) for f in html_files]
dated_files = [pair for pair in dated_files if pair[0] is not None]

dated_files.sort()
earliest_file = dated_files[0][1]
earliest_path = os.path.join(folder_path, earliest_file)

with open(earliest_path, 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')
    fomc_text = soup.get_text(separator=' ', strip=True)

prompt = f"""
You are a financial analyst specializing in monetary policy communications. Read the following FOMC announcement and answer the following questions:

1. What is the release date of the announcement?
2. What is the sentiment at the beginning of the announcement? (Bearish, Neutral, Bullish)
3. What is the sentiment at the end of the announcement?
4. Did the tone shift (e.g., Bearish to Bullish)?
5. Briefly explain the tone and any shift.
6. Provide a final numerical sentiment rating to the entire document based on your analysis of the tone shift and sentiment throughout, using this scale:
    -1 = Bearish
     0 = Neutral
     1 = Bullish
     The rating can be a decimal (e.g., -0.5, 0.75, etc.). 

Just output the rating number without any explanation.

FOMC Text:
{fomc_text}
"""

response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=500,
    temperature=0.4
)

response_text = response.choices[0].message.content
print("Full Response Text:")
print(response_text)

match = re.search(r"6\.\s*(-?\d*\.\d+|\d+)", response_text)
if match:
    extracted_rating = float(match.group(1))
    print(f"Extracted Rating: {extracted_rating}")
else:
    print("No rating found")

Full Response Text:
1. The release date of the announcement is February 2, 2000.
2. The sentiment at the beginning of the announcement is Neutral.
3. The sentiment at the end of the announcement is Bearish.
4. Yes, the tone did shift from Neutral to Bearish.
5. The tone at the beginning of the announcement is neutral as the FOMC simply states its decision to raise the federal funds rate and the discount rate. However, the tone shifts to bearish as the FOMC expresses its concern about potential inflationary imbalances that could harm the economy's expansion. The FOMC believes that the risks are mainly towards conditions that may generate heightened inflation pressures in the foreseeable future.
6. -0.5
Extracted Rating: -0.5


### ChronoBERT?!?!?!?!?!?!?!?